In [19]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
# from sklearn import 

In [33]:
dataset1 = pd.read_csv('data/dataset1_v1.csv')
dataset1.label.replace(-1,0,inplace=True)
dataset2 = pd.read_csv('data/dataset2_v1.csv')
dataset2.label.replace(-1,0,inplace=True)
dataset3 = pd.read_csv('data/off_test_v1.csv')

In [35]:
dataset1[dataset1.label==-1]
dataset1.shape

(136301, 56)

In [37]:
dataset1.drop_duplicates(inplace=True)
dataset2.drop_duplicates(inplace=True)
dataset3.drop_duplicates(inplace=True)

dataset12 = pd.concat([dataset1,dataset2],axis=0)
dataset12.shape

(386897, 56)

In [22]:
dataset1_y = dataset1.label #标签
#特征集
dataset1_x = dataset1.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)  # 'day_gap_before','day_gap_after' cause overfitting, 0.77

dataset2_y = dataset2.label
dataset2_x = dataset2.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

dataset12_y = dataset12.label
dataset12_x = dataset12.drop(['user_id','label','day_gap_before','day_gap_after'],axis=1)

dataset3_preds = dataset3[['user_id','coupon_id','date_received']]
dataset3_x = dataset3.drop(['user_id','coupon_id','date_received','day_gap_before','day_gap_after'],axis=1)

In [23]:
dataset1_x.shape,dataset2_x.shape,dataset3_x.shape

((134416, 52), (252481, 52), (112803, 52))

In [24]:
dataset1 = xgb.DMatrix(dataset1_x,label=dataset1_y)
dataset2 = xgb.DMatrix(dataset2_x,label=dataset2_y)
dataset12 = xgb.DMatrix(dataset12_x,label=dataset12_y)
dataset3 = xgb.DMatrix(dataset3_x)

In [30]:
dataset1.info()

AttributeError: 'DMatrix' object has no attribute 'info'

In [25]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
}

In [26]:
watchlist = [(dataset12,'train')]
model = xgb.train(params,dataset12,num_boost_round=3500,evals=watchlist)

#predict test set
dataset3_preds['label'] = model.predict(dataset3)
dataset3_preds.label = MinMaxScaler().fit_transform(dataset3_preds.label)
dataset3_preds.sort_values(by=['coupon_id','label'],inplace=True)
dataset3_preds.to_csv("xgb_preds.csv",index=None,header=None)
dataset3_preds.describe()
    
#save feature score
feature_score = model.get_fscore()
feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
fs = []
for (key,value) in feature_score:
    fs.append("{0},{1}\n".format(key,value))
    
# with open('xgb_feature_score.csv','w') as f:
#     f.writelines("feature,score\n")
# f.writelines(fs)

[0]	train-auc:0.83007
[1]	train-auc:0.836854
[2]	train-auc:0.845115
[3]	train-auc:0.847762
[4]	train-auc:0.848959
[5]	train-auc:0.850138
[6]	train-auc:0.851078
[7]	train-auc:0.851949
[8]	train-auc:0.852407
[9]	train-auc:0.852479
[10]	train-auc:0.852743
[11]	train-auc:0.852699
[12]	train-auc:0.852794
[13]	train-auc:0.852742
[14]	train-auc:0.852503
[15]	train-auc:0.85254
[16]	train-auc:0.852205
[17]	train-auc:0.852405
[18]	train-auc:0.852466
[19]	train-auc:0.852511
[20]	train-auc:0.852354
[21]	train-auc:0.852621
[22]	train-auc:0.852457
[23]	train-auc:0.852346
[24]	train-auc:0.852538
[25]	train-auc:0.852987
[26]	train-auc:0.853264
[27]	train-auc:0.853303
[28]	train-auc:0.853537
[29]	train-auc:0.853361
[30]	train-auc:0.853444
[31]	train-auc:0.853895
[32]	train-auc:0.854011
[33]	train-auc:0.854109
[34]	train-auc:0.854113
[35]	train-auc:0.854203
[36]	train-auc:0.85423
[37]	train-auc:0.854348
[38]	train-auc:0.85444
[39]	train-auc:0.854445
[40]	train-auc:0.854441
[41]	train-auc:0.854507
[42]	t

d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
d:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [28]:
with open('xgb_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(fs)
